In [ ]:
import numpy as np
import pandas as pd
from src import BayesRegressionUpdate, BayesVarianceUpdate, mvnrndT, RandStdNormT, qDraw

from roll_gibbs import roll_gibbs

import seaborn as sns
import matplotlib.pyplot as plt

# Assume roll_gibbs function has been defined elsewhere as per your earlier request.

# Set parameters
INFINITY = 1e30
eps = 1e-30

# Generate some price data
nObs = 30
sdu = 0.01
c = 0.01
np.random.seed(345699)  # for repeatability"

m_values = sdu * np.random.normal(size=nObs).cumsum()
q_values = 1 - 2 * (np.random.uniform(size=nObs) > 0.5)
p_values = m_values + c * q_values

sim = pd.DataFrame({'q': q_values, 'p': p_values})

In [ ]:
# Simulate parameters and q's
nSweeps = 10000

_qDraw=True
varuDraw=True 
regDraw=True,
varuStart=sdu**2 
cStart=c 
printLevel=0 
cLower=-INFINITY 
cUpper=INFINITY

np.random.seed(1234)  # Initialize the random number generators
dsIn = sim.copy()
# Read in data
# Assuming dsIn is a pandas DataFrame
if _qDraw:
    p = dsIn['p'].values
    q = np.empty(p.shape)
else:
    p = dsIn['p'].values
    q = dsIn['q'].values

nObs = len(p)

# Initialize output data (will be filled and then converted to pandas DataFrame later on)
qOut_data = {'sweep': [], 't': [], 'q': []}
parmOut_data = {'sweep': [], 'sdu': [], 'c': []}

dp = p[1:] - p[:-1]

if _qDraw:
    qInitial = np.concatenate(([1], np.sign(dp)))
    qInitial[qInitial == 0] = 1  # Only initialize nonzero elements of q
    q = qInitial

varu = varuStart
c = cStart

dq = q[1:] - q[:-1]
dp = np.matrix(dp).T
dq = np.matrix(dq).T

for sweep in range(1, nSweeps + 1):
    if sweep % 1000 == 0:
        print(f'Sweep: {sweep}')

    if regDraw:
        priorMu = np.array([[0]])
        priorCov = np.array([[1]])
        postMu, postCov = BayesRegressionUpdate(priorMu, priorCov, dp, dq, varu)
        if printLevel >= 2:
            print(postMu, postCov)
        N = postMu.shape[0] * postMu.shape[1]
        if isinstance(cLower, float) and isinstance(cUpper, float):
            clower = np.matrix([cLower]*N).T
            cupper = np.matrix([cUpper]*N).T
        c = mvnrndT(postMu, postCov, clower, cupper)
    
        if printLevel >= 2:
            print(c)

    if varuDraw:
        if c.shape[0] * c.shape[1] == 1 : 
            u = dp - c[0,0] * dq
        else : 
            u = dp - c * dq
        
        priorAlpha = 1e-12
        priorBeta = 1e-12

        postAlpha, postBeta = BayesVarianceUpdate(priorAlpha, priorBeta, u)
        x = (1 / postBeta) * np.random.gamma(postAlpha)
        varu = 1 / x
        sdu = np.sqrt(varu)
        if printLevel >= 2:
            print(varu)

    if _qDraw:
        qDrawPrintLevel = 0
        if c.shape[0] * c.shape[1] == 1:
            qDraw(p, q, c[0,0], varu, qDrawPrintLevel) 
        else:
            raise ValueError('c is not a scalar')
        # Collect output data
        qOut_data['sweep'].extend([sweep] * nObs)
        qOut_data['t'].extend(range(1, nObs + 1))
        qOut_data['q'].extend(q)

    if regDraw or varuDraw:
        parmOut_data['sweep'].append(sweep)
        parmOut_data['sdu'].append(sdu)
        if c.shape[0] * c.shape[1] == 1:
            parmOut_data['c'].append(c[0,0])
        else:
            raise ValueError('c is not a scalar')

# Convert dictionaries to pandas DataFrames and save (if necessary)
qOut_df = pd.DataFrame(qOut_data)
parmOut_df = pd.DataFrame(parmOut_data)

In [ ]:
parmOut_df.describe()